In [ ]:
# Implement naive/BM algorithms that count and return 
    # of character comparisons performed
    # of alignments tried

In [ ]:
# Naive
def naive_with_counts(p, t):
    occurrences = []
    char_comparisons = 0
    num_alignments = 0
    # Loop through alignments
    for i in range(len(t) - len(p) + 1):
        num_alignments += 1
        match = True
        # Loop through characters
        for j in range(len(p)):
            char_comparisons +=1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
          occurrences.append(i)
    return occurrences, num_alignments, char_comparisons

p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# Boyer-Moore
from bm_preproc import BoyerMoore

def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    char_comparisons = 0
    num_alignments = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            char_comparisons +=1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, char_comparisons

p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# Quiz

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('chr1.GRCh38.excerpt.fasta')
test = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

# 1: How many alignments does naive try w/ test? - 799954
# 2: How many character comparisons? - 984143
occurrences, num_alignments, num_character_comparisons = naive_with_counts(test, genome)
print num_alignments, num_character_comparisons

# 3: How many aligments does BM try? - 127974 165191
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(test)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(test, p_bm, genome)
print num_alignments, num_character_comparisons

In [ ]:
import bisect 
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets

def approximate_match(p, t, n):
    index = Index(t, 8)
    index_hits = 0
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    for i in range(n+1):
        start = i*segment_length
        # in case you have a weird end
        end = min((i+1)*segment_length, len(p))
        matches = queryIndex(p[start:end], t, index)
        index_hits += len(matches)
        #p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        #matches = boyer_moore(p[start:end], p_bm, t)
        
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), index_hits



p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = readGenome('chr1.GRCh38.excerpt.fasta')
hits, index_hits = approximate_match(p, t, 2)
# 4 - 19
print len(hits)
# 5 - 90
print index_hits
# Implement PP using Index to find exact mathces for partitions
# P always has length 24, looking for approx matches with 2 mismatches, 8-mer index

In [53]:
# 6
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def query_subseq(p, t, index):
    k = index.k
    offsets = []
    num_index_hits = 0
    for j in range(0, 3):
        for i in index.query(p[j:]):
            num_index_hits += 1
            if p[k:] == t[i+k:i+len(p)]:
                offsets.append(i)
    return offsets, num_index_hits

t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)


occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(occurrences, num_index_hits)

t = open('1110.txt.utf-8').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)

occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(occurrences)
print(num_index_hits)
# Write a function that, given a length-24 pattern P 
# and given a 𝚂𝚞𝚋𝚜𝚎𝚚𝙸𝚗𝚍𝚎𝚡 object built with k = 8 and ival = 3 
# finds all approximate occurrences of P within T with up to 2 mismatches.

([0, 14], 6)
[135249]
3


In [55]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = readGenome('chr1.GRCh38.excerpt.fasta')
subseq_ind = SubseqIndex(t, 8, 3)

occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(occurrences)
print(num_index_hits)

[56922, 262042, 273669, 364263, 465647, 657496, 717706]
79


In [ ]:
# ANSWERS:
# 1: 799954
# 2: 984143
# 3: 127974
# 4: 19
# 5: 90
# 6: 79